# STA 663 Final Project #
## Implementation of FFSR in Python ##
### Nicole Solomon ###

## 1. Introduction

Fast false selection rate (FFSR) is a model selection method developed by Boos, Stefanski, and Wu (Boos, 2009). 
This method is a variable selection, model selection, and model-sizing tool under the context of forward selection. These three aspects are determined by controlling for the 'false selection rate' (FSR) or the rate at which unimportant variables are added to the model.
The original FSR method developed by the same authors relies on simulating 'phony' variables (unrelated to the outcome) 
and computing the rate at which these enter the model in order to determine the appropriate '$\alpha$-to-enter' level; i.e. 
the upper threshold on p-value significance of a variable to be entered into the model. This $\alpha$ level is selected so as 
to control the average rate of inclusion of meaningless variables in a forward selection approach. The Fast FSR (FFSR) 
method improves upon this approach by approximating the rate of inclusion of uninformative variables with $\alpha$ rather than 
on a term dependent on bootstrapping the simulated phony variables. This has been shown to have results very similar to 
that of FSR and is particularly efficient with bagging for the purpose of model averaging and prediction (Boos et al, 2009). This technique has applications in Cox, logistic, and linear regression.

The goal of this project was to implement this algorithm under linear regression in Python. Specifically, functions were be written for the Fast FSR technique for three contexts: in its simplest form for pure variable selection, allowing for forced inclusion of a subset of variables, 
and with bagging. The code was tested on a 
dataset available from NCSU in order to demonstrate the technique 
and its efficiency. The algorithm was also compared to an equivalent R version of the algorithm (Boos, ncsu.edu) on the same dataset to 
demonstrate the correctness of the Python function.

## 2. Implementation & Pseudocode

Traditional forward selection can be slow in sizeable datasets.  In order to produce an efficient algorithm, the 
`regsubsets` function from the `leaps` package in `R` was utilized to conduct the forward selection 
aspect of the FFSR algorithm.  This function utilizes a branch-and-bound technique to quickly identify optimal subsets in 
the linear prediction of y by x, rather than fitting all possible models at each iteration (Lumley, 2015).
The remainder of the algorithm was implemented in a modular programming fashion in Python.  These modules perform the 
following tasks:

### 2.1 Pseudocode
    1. Perform forward selection via the `regsubsets` function called in Python via the RPY2 library
	2. Store covariate order of entry into the model
	3. Compute p-values for each sequential covariate entering the model
    4. Monotonize p-values by taking sequential max
	5. Compute gamma values for each step in model size
	6. Compute alpha values for each monotonized p-value
	7. Compute alpha for a pre-specified gamma (optional)
	8. Estimate parameters for the final fitted model

Additional functions were written to neatly compile the results, as well as check for appropriate data input type.
All of these functions are called within the primary 'ffsr' function.  A streamlined version of this function was written 
without the data check or table compilation for the sake of bagging.  An additional 'bag_fsr' function was written for 
implementation of FFSR with bagging; this function iteratively runs the ffsr algorithm on a duplicate of the original data 
built from randomly selecting the original rows with replacement.  In this manner the resulting parameter estimates can be 
averaged, akin to model averaging.  This produces predictions more accurate than those obtained with just one application 
of the FFSR algorithm (Boos, 2009).

## 3. Testing, Profiling, Optimization

Unit tests were drafted to assure that functions failed when appropriate, and raise 
specific warnings.  The functions passed all unit tests as seens below.  The code was also profiled to allow for optimization to improve efficiency and speed.  The primary 
bottleneck in the primary ffsr function is within the forward selection modular function.  This procedure requires more 
than 75% of the time necessary to run the functions.  Utilizing an `R` function rather than a Python or C function 
appears to be the reason for the slow speed.  This issue is addressed in more detail in the next section.

In [45]:
import os
os.chdir('/home/bitnami/STA-663-Nicole-Solomon-Project/Tests')
!py.test

============================= test session starts ==============================
platform linux2 -- Python 2.7.9 -- py-1.4.25 -- pytest-2.6.3
collected 33 items 

test_alpha.py ......
test_alphag.py ..........
test_bagfsr.py ...
test_beta.py .....
test_covnames.py .
test_df_type.py ..
test_ffsr.py ..
test_gamma.py ...
test_pvals.py .

========================== 33 passed in 2.51 seconds ===========================


## 4. Application and Comparison

The Python algorithm was applied to a NCAA basketball dataset obtained from NCSU (Boos, ncsu.edu).  The R FFSR algorithm was 
also applied to this data in order to compare the efficiency of the Python algorithm.

### 4.1 Standard FFSR function

In [35]:
%run ffsr_r_run
%run -t -m ffsr_p_run

NCSU R Results:

   var   pval  pvmax    Rsq      g
1    2 0.0000 0.0000 0.7069 0.0000
2    3 0.0001 0.0001 0.7539 0.0004
3    5 0.0116 0.0116 0.7708 0.0270
4    4 0.0053 0.0116 0.7901 0.0270
5    7 0.0025 0.0116 0.8110 0.0270
6   17 0.0433 0.0433 0.8197 0.0804
7   15 0.0527 0.0527 0.8274 0.0791
8    6 0.1056 0.1056 0.8327 0.0864
9    9 0.0826 0.1056 0.8386 0.0864
10   8 0.0536 0.1056 0.8457 0.0864
11  12 0.2350 0.2350 0.8484 0.1566
12  10 0.2864 0.2864 0.8505 0.1542
13  13 0.3163 0.3163 0.8524 0.1054
14  18 0.2697 0.3163 0.8546 0.1054
15  11 0.4953 0.4953 0.8555 0.1238
16   1 0.6326 0.6326 0.8559 0.1116
17  14 0.7056 0.7056 0.8562 0.0784
18  19 0.8605 0.8605 0.8563 0.0453
19  16 0.9032 0.9032 0.8563 0.0000
   user  system elapsed 
  0.008   0.000   0.009 

Python Results:

     S  Var       p     p_m alpha_F gamma_F
0    1   x2  0.0000  0.0000  0.0056  0.0000
1    2   x3  0.0001  0.0001  0.0088  0.0004
2    3   x5  0.0116  0.0116  0.0125  0.0270
3    4   x4  0.0053  0.0116  0.0167  0.

### 4.2 FFSR function with forced in variables

In [37]:
%run ffsr_force_r_run
%run -t -m ffsr_force_p_run

NCSU R Results:

   var   pval  pvmax    Rsq
1   12 0.0000 0.0000 0.6042
2    3 0.0000 0.0000 0.6783
3    5 0.0000 0.0000 0.6874
4    2 0.0000 0.0000 0.7710
5    4 0.0043 0.0043 0.7914
6    7 0.0028 0.0043 0.8118
7   17 0.0539 0.0539 0.8198
8   15 0.0458 0.0539 0.8281
9    6 0.0976 0.0976 0.8336
10   9 0.0962 0.0976 0.8391
11   8 0.0281 0.0976 0.8484
12  10 0.2864 0.2864 0.8505
13  13 0.3163 0.3163 0.8524
14  18 0.2697 0.3163 0.8546
15  11 0.4953 0.4953 0.8555
16   1 0.6326 0.6326 0.8559
17  14 0.7056 0.7056 0.8562
18  19 0.8605 0.8605 0.8563
19  16 0.9032 0.9032 0.8563
   user  system elapsed 
  0.007   0.000   0.006 

Python Results:

     S  Var       p     p_m alpha_F gamma_F
0    1  x12  0.0000  0.0000  0.0056  0.0000
1    2   x3  0.0000  0.0000  0.0088  0.0001
2    3   x5  0.1100  0.1100  0.0125  0.0733
3    4   x2  0.0000  0.1100  0.0167  0.0733
4    5   x4  0.0043  0.1100  0.0214  0.0733
5    6   x7  0.0028  0.1100  0.0269  0.0733
6    7  x17  0.0539  0.1100  0.0333  0.0733
7  

### 4.3 FFSR with bagging

In [38]:
%run ffsr_bag_r_run
%run -t -m ffsr_bag_p_run

NCSU R Results:

   user  system elapsed 
  0.735   0.005   0.747 

[1] "Mean of estimated alpha-to-enter: 0.0445"

[1] "Mean size of selected model: 7.695"

Python Results:


Mean of estimated alpha-to-enter: 0.0503

Mean size of selected model: 7.095

IPython CPU timings (estimated):
  User   :       6.12 s.
  System :       0.27 s.
Wall time:       5.50 s.


As seen in the results above, the Python algorithm yields results identical to those returned by the R algorithm.  The bagging results differ somewhat due to the random resampling in Python and R that cannot be matched via setting identical seeds.  These results overall demonstrate the validity of the Python FFSR function.  However, the Python algorithm is significantly slower than the R function.  This is due to the 
overhead time spent in calling R, running the R function `regsubsets`, and then returning the results for processing.  
This procedure would be significantly faster and competitive with the equivalent R algorithm if the Fortran code upon which 
`regsubsets` is based were to be wrapped in C and thereby made directly callable in Python.  This is a daunting task 
however as the original Fortran code is of the type Fortran 77 and was written decades ago (Lumley, github.com).  At the very 
least, an alternative, but performance-wise competitive, forward selection procedure is necessary to improve the Python 
algorithm beyond its current speed.

## References

1. Boos D, Stefanski L, and Wu Y. "Fast FSR Variable Selection with Applications to Clinical Trials." _Biometrics_. 2009; 65(3): 692-700.
2. Boos D and Stefanski L. "Fast FSR: Controlling the False Selection Rate without Simulation." NCSU. http://www4.stat.ncsu.edu/~boos/var.select/fast.fsr.html.
3. Thomas Lumley. "Package 'leaps'". R Foundation for Statistical Computing, 2015. Version 2.9.
4. Thomas Lumley and Alan Miller. "cran/leaps." https://github.com/cran/leaps/tree/master/src.
